In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [10]:
image_dir = "flower_images"
all_images = []
numberOffiles = 0
for filename in os.listdir(image_dir):
    if filename.lower().endswith((".png")):
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0 #normalize
        all_images.append(img)
        numberOffiles += 1

print("numberOffiles = ", numberOffiles)

numberOffiles =  210


In [11]:
base_model = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
flower_images = np.array(all_images)
flower_images_preprocessed = preprocess_input(flower_images)
features = base_model.predict(flower_images_preprocessed)
features = features.reshape((210, 7*7*512))
features.shape
#https://talkai.info/chat/


7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step


(210, 25088)

In [24]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

# Load the labels from the CSV file
labels_df = pd.read_csv('flower_images/flower_labels.csv')
true_labels = labels_df.iloc[:,1].values  # Assuming the second column contains the labels
# Train the KMeans model
# kmeans = KMeans(n_clusters=len(np.unique(true_labels)), random_state=42)
kmeans = KMeans(n_clusters=7, random_state=42)
predicted_labels = kmeans.fit_predict(features)

# Map the predicted labels to the true labels
# This is necessary because KMeans is unsupervised and doesn't know the label correspondence
label_mapping = {}
for i in range(len(np.unique(true_labels))):
    mask = (true_labels == i)
    label_mapping[i] = np.bincount(predicted_labels[mask]).argmax()

mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# Calculate the accuracy
accuracy = accuracy_score(true_labels, mapped_predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 19.05%


In [16]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Assuming `features` is your (210, 7*7*512) matrix from the VGG16 model
# You will need to choose appropriate values for eps and MinPts
# These values are highly dependent on your data and the scale of the features
dbscan = DBSCAN(eps=0.5, min_samples=5)

# Fit the model
dbscan.fit(features)

# Get the cluster labels (assigned to each data point)
cluster_labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present
n_clusters_ = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise_ = list(cluster_labels).count(-1)

print(f'Estimated number of clusters: {n_clusters_}')
print(f'Estimated number of noise points: {n_noise_}')

# Calculate the silhouette score to evaluate the quality of the clustering
silhouette_avg = silhouette_score(features, cluster_labels)
print(f'Silhouette Coefficient: {silhouette_avg:.2f}')


Estimated number of clusters: 0
Estimated number of noise points: 210


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)